## Part 1: 搭建一个分词工具

### Part 1.1  基于枚举方法来搭建中文分词工具

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率 word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 对于给定字符串：”我们学习人工智能，人工智能是未来“, 找出所有可能的分割方式
- [我们，学习，人工智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工智能，是，未来]
- [我们，学习，人工，智能，人工，智能，是，未来]
- [我们，学习，人工智能，人工，智能，是，未来]
.......


#### Step 2: 我们也可以计算出每一个切分之后句子的概率
- p(我们，学习，人工智能，人工智能，是，未来)= -log p(我们)-log p(学习)-log p(人工智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工智能)-log p(是)-log p(未来)
- p(我们，学习，人工，智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工)-log p(智能)-log p(人工)-log p(智能)-log p(是)-log p(未来)
- p(我们，学习，人工智能，人工，智能，是，未来)=-log p(我们)-log p(学习)-log p(人工智能)-log p(人工)-log p(智能)-log(是)-log p(未来)
.....

#### Step 3: 返回第二步中概率最大的结果

In [ ]:
# TODO: 第一步： 现代汉语常用词表.txt中读取所有中文词及对应频率。
from collections import Counter
import random
import numpy as np

with open('./zidian.txt', encoding='utf-8') as f:
    lines = f.readlines()

sum = 0
# counter是一个python中的计数器，继承于dic类，因此可以使用dic的方法
word_prob = Counter()

for line in lines:
    columns = line.strip().split()
    # 重复词频率直接相加，（相同词多次出现是因为发音不同，即语义也不同，这里不做区分）
    # 注意，这里columns[-1]是每一行的最后一个元素，是前面的词出现的频率
    # column[1]第一个元素是各行的单词，但是这个词典里发音不同的算两个，我们这里简化，不区分不同发音的词语，顾当column[1]相同时将他们的数字相加
    # 经过外层的for循环之后，所有的元素就会添加进counter这个计数器中
    word_prob[columns[0]] += int(columns[-1])
    sum += int(columns[-1])

# 频率转为概率
for word in word_prob:
    word_prob[word] /= sum
    

In [ ]:
# 测试
print([{word: word_prob[word]} for word in random.sample(word_prob.keys(), 3)])
print("词典大小：%d" % len(word_prob))
print(np.sum(list(word_prob.values())))

In [ ]:
#  分数（10）
## TODO 请编写word_segment_naive函数来实现对输入字符串的分词
def sentence_break(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    # 注意 是基于枚举的方法，不考虑正向最大匹配
    # max_len = 0 # 获取最大分词长度
    # for w in dic_words:
    #     if len(w) > max_len:
    #         max_len = len(w)
    # 
    # TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。
    # 定义一个空列表来储存分词结果
    # result_word = []
    # k = 0 # 切割输入字符串从下标0开始
    # 
    # while k < len(input_str):
    #     test_str = input_str[k: max_len+k]
    #     if test_str in dic_words:
    #         result_word.append(test_str)
    #         k = max_len + k
    #     else:
    #         while len(test_str) > 1:
    #             test_str = test_str[0:len(test_str)-1]
    #             if test_str in dic_words or len(test_str) == 1:
    #                 result_word.append(test_str)
    #                 k = len(test_str) - 1 + k
    #             k = len(test_str) - 1
    # return result_word
                        
    """重新考虑基于枚举的方法, 首先要求所有的分词结果，就是求该句话在字典下的全切分。
    """
    # 定义一个空间暂时储存排列
    mem = [[] for _ in range(len(input_str))]
    
    import math
    for i in range(0, len(input_str)):
        for j in range(0, i+1): # 从一个字节开始，考虑每一个词是否在字典中存在
            if j == 0: # 如果刚刚取出的字符串就在字典中则加入mem
                if input_str[j:i+1] in word_prob:
                    mem[i].append([input_str[j:i+1]])
                continue # 继续循环，减少字符串长度看在不在字典中
            
            # 当取出的字符串不在其中时，就要根据前一个状态往下推，要确定依赖的前状态存在且达成转移条件
            if mem[j-1] and input_str[j:i+1] in word_prob: 
                for status in mem[j-1]: # 对应前状态的每一个可能加上现在新出现的词
                    mem[i].append(status + [input_str[j:i+1]])
    # 返回最后一个mem的储存值就是基于字典的全切分
    segments = [mem[-1]]
    return segments

In [ ]:
print(sentence_break("北京的天气真好啊"))
print(sentence_break("今天的课程内容很有意思"))
print(sentence_break("经常有意见分歧"))

In [5]:
# 存储所有分词的结果。如果次字符串不可能被完全切分，则返回空列表(list)
    # 格式为：segments = [["今天"，“天气”，“好”],["今天"，“天“，”气”，“好”],["今“，”天"，“天气”，“好”],...]
import math   
    # TODO: 第二步：循环所有的分词结果，并计算出概率最高的分词结果，并返回
def word_segment_naive(input_str):
    """
    1. 对于输入字符串做分词，并返回所有可行的分词之后的结果。
    2. 针对于每一个返回结果，计算句子的概率
    3. 返回概率最高的最作为最后结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    # TODO： 第一步： 计算所有可能的分词结果，要保证每个分完的词存在于词典里，这个结果有可能会非常多。 
    segments = sentence_break(input_str)
    best_segment = list()
    best_score = 0
    for seg in segments:
        # TODO ...
        if seg:
            score = 0
            for word in seg:
                score += word_prob[str(word)]
            if best_score == 0:
                best_segment = seg
                best_score = score
            else:
                if score > best_score:
                    best_segment = seg
                    best_score = score
    
    return best_segment
     

In [6]:
# 测试
print(word_segment_naive("北京的天气真好啊"))
print(word_segment_naive("今天的课程内容很有意思"))
print(word_segment_naive("经常有意见分歧"))

[['北京', '的', '天气', '真', '好', '啊'], ['北', '京', '的', '天气', '真', '好', '啊'], ['北京', '的', '天', '气', '真', '好', '啊'], ['北', '京', '的', '天', '气', '真', '好', '啊']]
[['今天', '的', '课程', '内容', '很', '有意思'], ['今', '天', '的', '课程', '内容', '很', '有意思'], ['今天', '的', '课', '程', '内容', '很', '有意思'], ['今', '天', '的', '课', '程', '内容', '很', '有意思'], ['今天', '的', '课程', '内', '容', '很', '有意思'], ['今', '天', '的', '课程', '内', '容', '很', '有意思'], ['今天', '的', '课', '程', '内', '容', '很', '有意思'], ['今', '天', '的', '课', '程', '内', '容', '很', '有意思'], ['今天', '的', '课程', '内容', '很', '有', '意思'], ['今', '天', '的', '课程', '内容', '很', '有', '意思'], ['今天', '的', '课', '程', '内容', '很', '有', '意思'], ['今', '天', '的', '课', '程', '内容', '很', '有', '意思'], ['今天', '的', '课程', '内', '容', '很', '有', '意思'], ['今', '天', '的', '课程', '内', '容', '很', '有', '意思'], ['今天', '的', '课', '程', '内', '容', '很', '有', '意思'], ['今', '天', '的', '课', '程', '内', '容', '很', '有', '意思'], ['今天', '的', '课程', '内容', '很', '有意', '思'], ['今', '天', '的', '课程', '内容', '很', '有意', '思'], ['今天', '的', '课', '程', '内容', '很', '有意', '

### Part 1.2  基于维特比算法来优化上述流程

此项目需要的数据：
1. 综合类中文词库.xlsx： 包含了中文词，当做词典来用
2. 以变量的方式提供了部分unigram概率word_prob


举个例子： 给定词典=[我们 学习 人工 智能 人工智能 未来 是]， 另外我们给定unigram概率：p(我们)=0.25, p(学习)=0.15, p(人工)=0.05, p(智能)=0.1, p(人工智能)=0.2, p(未来)=0.1, p(是)=0.15

#### Step 1: 根据词典，输入的句子和 word_prob来创建带权重的有向图（Directed Graph） 参考：课程内容
有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率已经给出（存放在word_prob）。
注意：思考用什么方式来存储这种有向图比较合适？ 不一定只有一种方式来存储这种结构。 

#### Step 2: 编写维特比算法（viterebi）算法来找出其中最好的PATH， 也就是最好的句子切分
具体算法参考课程中讲过的内容

#### Step 3: 返回结果
跟PART 1.1的要求一致

In [8]:
## TODO 编写word_segment_viterbi函数来实现对输入字符串的分词
import math

def word_segment_viterbi(input_str):
    """
    1. 基于输入字符串，词典，以及给定的unigram概率来创建DAG(有向图）。
    2. 编写维特比算法来寻找最优的PATH
    3. 返回分词结果
    
    input_str: 输入字符串   输入格式：“今天天气好”
    best_segment: 最好的分词结果  输出格式：["今天"，"天气"，"好"]
    """
    
    # TODO: 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph）
    #      有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率在 word_prob，如果不在word_prob里的单词但在
    #      词典里存在的，统一用概率值1e-100。
    # 图是为了直观起见，边表示字或词及其概率，节点存储状态，图有没有其实无所谓，从本质上讲其实就是个状态转移算法
    # 每个节点的状态包含-log(P)和当前最优切分
    """储存是列表的列表，第一个元素是概率，第二个是路径"""
    memory = [[0, []] for _ in range(len(input_str)+1)]
    
    # TODO： 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    # TODO: 第三步： 根据最好的PATH, 返回最好的切分
    for i in range(1, len(input_str)+1):
        for j in range(i):
            # 这里偷个懒，默认没有形成词的单字可以在词典中找到（如果不成立事实上会返回完整句子，因为-log(1e-100)必然小于该值加某个非负数
            """这里是把各个长度的词考虑进来，参考最大正向匹配"""
            word = input_str[j:i]
            """如果在字典中就取其概率，不在里面就取1e-100"""
            prob = word_prob[word] if word in word_prob else 1e-100 
            
            """取对数是为了防止下溢？"""
            score = memory[j][0] - math.log(prob)
            # 状态更新
            if memory[i][0] == 0: # 如果概率为0则把求出的score赋值给他
                memory[i][0] = score 
                memory[i][1] = memory[j][1] + [word] # 把现在的字符串加入到路径后面
            else:
                if score < memory[i][0]: # 将分数更高的路径分数赋值给score，并添加新的路径
                    memory[i][0] = score 
                    memory[i][1] = memory[j][1] + [word]

    return memory[-1][1], memory[-1][0] # 返回倒数第一个列表的第二个元素，就是最优路径，返回第一个元素就是概率

In [9]:
# 测试
print(word_segment_viterbi("北京的天气真好啊"))
print(word_segment_viterbi("今天的课程内容很有意思"))
print(word_segment_viterbi("经常有意见分歧"))


(['北京', '的', '天气', '真', '好', '啊'], 89.71108041568827)
(['今天', '的', '课程', '内容', '很', '有意思'], 95.63149358104688)
(['经常', '有意', '见', '分歧'], 55.89707633882324)
